#### Model4: ex-Gaussian

我们假定反应时是混合高斯分布

![Image Name](https://docs.pymc.io/en/latest/_images/pymc-ExGaussian-1.png)

图片来源：https://docs.pymc.io/en/latest/api/distributions/generated/pymc.ExGaussian.html

In [7]:
with pm.Model() as exgaussian:
    # 先验分布:alpha,beta,sigma,nu这三个参数是随机变量
    alpha = pm.HalfNormal('alpha', sd=1)
    beta = pm.HalfNormal('beta', sd=1)
    sigma = pm.HalfNormal('sigma', sd=1) 
    nu = pm.HalfNormal('nu', sd=1)   
    # 自变量conf是之前已经载入的数据
    x = pm.Data("x", data['conf'])
    # 参数mu是确定性随机变量，这个变量的值完全由右端值确定
    mu = pm.Deterministic("mu",  alpha+ beta*x) 

    # Y的观测值，这是一个特殊的观测随机变量，表示模型数据的可能性。也可以表示模型的似然，通过 observed 参数来告诉这个变量其值是已经被观测到了的，不会被拟合算法改变
    y_obs = pm.ExGaussian('y_obs',mu=mu,sigma=sigma,nu=nu,observed=data['rt'] )

In [8]:
with exgaussian:
    trace4_for_comp = pm.sample(draws = 2000, tune=1000, target_accept=0.9,chains=2, cores= 2)    
    # 将pymc的采样对象转化为inferencedata
    trace4=az.from_pymc3(trace4_for_comp)

WARNING (theano.tensor.opt): Optimization Warning: The Op erfcx does not provide a C implementation. As well as being potentially slow, this also disables loop fusion.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [nu, sigma, beta, alpha]


Sampling 2 chains for 1_000 tune and 2_000 draw iterations (2_000 + 4_000 draws total) took 39 seconds.


In [10]:
az.plot_trace(trace4, var_names=["alpha", "beta", "sigma","nu"]);

<Figure size 864x576 with 8 Axes>

In [11]:
pm.model_to_graphviz(exgaussian)

In [12]:
az.summary(trace4, var_names=["alpha", "beta", "sigma","nu"])

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha,0.667,0.012,0.644,0.689,0.0,0.0,1449.0,1568.0,1.0
beta,0.027,0.013,0.003,0.049,0.0,0.0,1059.0,553.0,1.0
sigma,0.183,0.010,0.165,0.201,0.0,0.0,1704.0,1987.0,1.0
nu,0.722,0.016,0.691,0.750,0.0,0.0,1827.0,2135.0,1.0


In [14]:
with exgaussian:
    # pm.sample_posterior_predictive()利用trace.posterior的后验分布计算后验预测分布
    ppc_y = pm.sample_posterior_predictive(trace4.posterior) 
#将ppc_y转化为InferenceData对象合并到trace中
az.concat(trace4, az.from_pymc3(posterior_predictive=ppc_y), inplace=True)

/opt/conda/lib/python3.7/site-packages/arviz/data/io_pymc3.py:100: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  FutureWarning,


In [15]:
# 绘制后验预测分布
az.plot_ppc(trace4)

<AxesSubplot:xlabel='y_obs'>

<Figure size 432x288 with 1 Axes>

In [23]:
# 将三个模型的采样结果进行比较
compare_dict = {"normal": trace, "log-nomal": trace2, "gamma": trace3,"exgaussian":trace4}
# 选择loo方法进行比较
comp = az.compare(compare_dict, ic='loo')
comp

/opt/conda/lib/python3.7/site-packages/arviz/stats/stats.py:146: UserWarning: The default method used to estimate the weights for each model,has changed from BB-pseudo-BMA to stacking
  "The default method used to estimate the weights for each model,"


,rank,loo,p_loo,d_loo,weight,se,dse,warning,loo_scale
log-nomal,0,-3520.817437,2.748285,0.000000,8.847524e-01,49.754464,0.000000,False,log
exgaussian,1,-3549.189661,3.758438,28.372225,1.152476e-01,50.415942,8.551291,False,log
gamma,2,-3596.401071,2.914570,75.583634,6.453105e-09,49.040403,8.502931,False,log
normal,3,-4102.216352,3.726850,581.398915,0.000000e+00,58.334996,29.515621,False,log


- 第一列为索引，它列出了传递给 az.compare(.) 的模型名称。

- rank 列：按照预测精度做的排名，值从0依次到模型总数，其中0代表最高精度。

- loo 列：各模型 ELPD 值的列表，总是按照 ELPD 值从最好到最差排序。

- p_loo 列：惩罚项的值列表，可以将其粗略地视为有效参数数量的估计值（但不要太认真）。此值可能低于具有更多结构的模型（如分层模型）中的实际参数数量，或者高于那些预测能力非常弱或严重错误指定的模型的实际参数数量。

- d_loo 列：每个模型与排名第一的模型之间的 LOO 相对差。因此第一个模型始终取值为0  。

- weight 列：分配给每个模型的权重。权重可以粗略地解释为在指定数据的条件下，是（参与比较的各模型中）该模型的概率。

- se 列：ELPD 的标准误差。

- dse 列：ELPD 相对差的标准误差。 dse 与 se 不一定相同，因为 ELPD 的不确定性在模型之间可能存在相关性。排名第一的模型 dse 值始终为0 。

- warning 列：如果为True，表示这是一个警告，LOO 的近似估计不可靠。

- loo_scale 列：估计值所用的尺度。默认为对数尺度。其他选项还包括：离差值尺度，即对数分值乘以-2，这会颠倒排序，较低的 ELPD 会更好；负对数尺度，即对数分值乘以-1，与离差值尺度一样，值越低越好。

In [24]:
# 将三个模型的采样结果进行比较
compare_dict = {"normal": trace, "log-nomal": trace2, "gamma": trace3,"exgaussian":trace4}
# 选择loo方法进行比较
comp = az.compare(compare_dict, ic='waic')
comp

/opt/conda/lib/python3.7/site-packages/arviz/stats/stats.py:146: UserWarning: The default method used to estimate the weights for each model,has changed from BB-pseudo-BMA to stacking
  "The default method used to estimate the weights for each model,"


,rank,waic,p_waic,d_waic,weight,se,dse,warning,waic_scale
log-nomal,0,-3520.817450,2.748299,0.000000,0.884752,49.754464,0.000000,False,log
exgaussian,1,-3549.189665,3.758442,28.372215,0.115248,50.415943,8.551290,False,log
gamma,2,-3596.401119,2.914618,75.583669,0.000000,49.040406,8.502933,False,log
normal,3,-4102.216048,3.726545,581.398598,0.000000,58.334952,29.515576,False,log


In [28]:
def comp_model(trace,model):
    '''
    trace: 未转化为inferencedata的采样结果
    model: 模型
    '''
    dftrc_m = pm.trace_to_dataframe(trace, include_transformed=True) # 将trace对象转化为dataframe
    trace = az.from_pymc3(trace) # 将trace对象转化为inferencedata对象
    trc_logp = trace['log_likelihood']['y_obs'].to_dataframe().groupby(['chain','draw']).sum().reset_index()['y_obs'] #从模型中提取对数似然
    
    #dic
    mean_deviance = -2 * trc_logp.mean(0) #计算所有参数对数似然的平均值
    deviance_at_mean = -2 * model.logp(dftrc_m.mean(0).to_dict()) #计算参数平均值的对数似然
    dic = 2 * mean_deviance - deviance_at_mean
    
    #bic
    deviance_at_mle = min(trc_logp) #对数似然最小的值
    parnum = 3 # 参数数量
    n = 3988 # 数据的样本数
    bic = -2 * deviance_at_mle +  2*parnum*np.log(n)
    
    #aic
    aic = -2 * deviance_at_mle +  2*parnum
    
    return dic,bic,aic

In [33]:
# 按照rank顺序计算各模型dic，bic，aic的值
dic1,bic1,aic1 = comp_model(trace=trace2_for_comp,model=LogNormal)
dic2,bic2,aic2 = comp_model(trace=trace4_for_comp,model=exgaussian)
dic3,bic3,aic3 = comp_model(trace=trace3_for_comp,model=Gamma)
dic4,bic4,aic4 = comp_model(trace=trace_for_comp,model=NormalModel)

/opt/conda/lib/python3.7/site-packages/arviz/data/io_pymc3.py:100: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/arviz/data/io_pymc3.py:100: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  FutureWarning,


In [34]:
model_comp = pd.DataFrame({ 
                        'rank':az.compare(compare_dict, ic='waic')['rank'],
                        'waic': az.compare(compare_dict, ic='waic')['waic'],
                        'loo': az.compare(compare_dict, ic='loo')['loo'],
                        'dic':[dic1,dic2,dic3,dic4],
                        'bic':[bic1,bic2,bic3,bic4],
                        'aic':[aic1,aic2,aic3,aic4],
                        })
model_comp

/opt/conda/lib/python3.7/site-packages/arviz/stats/stats.py:146: UserWarning: The default method used to estimate the weights for each model,has changed from BB-pseudo-BMA to stacking
  "The default method used to estimate the weights for each model,"
/opt/conda/lib/python3.7/site-packages/arviz/stats/stats.py:146: UserWarning: The default method used to estimate the weights for each model,has changed from BB-pseudo-BMA to stacking
  "The default method used to estimate the weights for each model,"
/opt/conda/lib/python3.7/site-packages/arviz/stats/stats.py:146: UserWarning: The default method used to estimate the weights for each model,has changed from BB-pseudo-BMA to stacking
  "The default method used to estimate the weights for each model,"


,rank,waic,loo,dic,bic,aic
log-nomal,0,-3520.817450,-3520.817437,7035.970548,7105.394937,7061.648666
exgaussian,1,-3549.189665,-3549.189661,7083.086535,7161.024899,7117.278628
gamma,2,-3596.401119,-3596.401071,7162.282507,7257.750778,7214.004507
normal,3,-4102.216048,-4102.216352,8196.515017,8269.588021,8225.841750
